# EDA and Data Visualization (Daily Trends)

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
# Load data for building 4
df = pd.read_csv("../data/preprocessed/Building_4.csv").astype({'datetime': 'datetime64'}).set_index('datetime')

In [7]:
df.head()

,daylight_savings,indoor_temp,avg_unmet_cooling_temp,indoor_hum,equipment_electric_power_kWh,dhw_heating_kWh,cooling_load_kWh,solar_generation_W_kW,outdoor_temp,outdoor_hum,...,pred_24h_outdoor_hum,pred_6h_diffuse_solar_W_m2,pred_12h_diffuse_solar_W_m2,pred_24h_diffuse_solar_W_m2,pred_6h_direct_solar_W_m2,pred_12h_direct_solar_W_m2,pred_24h_direct_solar_W_m2,holiday,workday,solar_generation_kW
datetime,,,,,,,,,,,,,,,,,,,,,
2008-01-02 00:00:00,0,22.02,0.0,55.86,11.7,3.00,1.61,0.0,9.40,94.12,...,91.65,-0.00,170.84,0.0,-0.00,5.32,0.0,True,False,0.0
2008-01-02 01:00:00,0,21.78,0.0,52.90,7.2,3.36,0.47,0.0,7.61,93.00,...,101.62,16.77,112.24,0.0,1.11,2.04,-0.0,True,False,0.0
2008-01-02 02:00:00,0,21.77,0.0,50.06,8.1,0.36,0.58,0.0,8.11,93.00,...,105.77,40.08,65.93,0.0,1.74,1.04,0.0,True,False,0.0
2008-01-02 03:00:00,0,21.81,0.0,47.95,12.4,0.12,1.19,0.0,8.36,93.00,...,92.80,65.63,38.59,-0.0,1.09,0.99,0.0,True,False,0.0
2008-01-02 04:00:00,0,21.77,0.0,46.91,8.6,0.00,0.59,0.0,8.78,93.00,...,100.62,107.26,15.92,0.0,1.78,0.84,0.0,True,False,0.0
